In [1]:
import json
import visualize
import os
import context as c, score
import pandas as pd

tokenizer = score.get_tokenizer()
dro_tokenizer = score.get_tokenizer("test-mlm/")

/home/gussuvmi@GU.GU.SE/venv/lib64/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load(file_name):
    with open(f"Results/raw/{file_name}", "r") as f:
        res = json.load(f)
    return res

# sdb
s_debiased = load("sdb_short.json")
l_debiased = load("sdb_long.json")
s_antonym = load("ant_short_probs.json")
l_antonym = load("ant_long_probs.json")

s_raw = visualize.get_sdb_df(s_debiased, c.context_t_i, tokenizer)
s_ant_raw = visualize.get_ant_prob_df(s_antonym, c.context_t_i)
l_raw = visualize.get_sdb_df(l_debiased, c.context_long_t_i, tokenizer)
l_ant_raw = visualize.get_ant_prob_df(l_antonym, c.context_long_t_i)

## SELF-DEBIAS

In [3]:
# means on word level
#import pandas 
#pandas.set_option('display.max_rows', 250)

In [4]:
# combine with antonyms

In [5]:
s_ant_comb = visualize.get_sdb_ant_df(s_raw, s_ant_raw)
l_ant_comb = visualize.get_sdb_ant_df(l_raw, l_ant_raw)

visualize.get_sdb_means(s_ant_comb, 'sdb_short_means.tex')
x= visualize.get_sdb_means(l_ant_comb, 'sdb_long_means.tex')

# get top n changes
visualize.get_top_n_changes(s_raw, file_name='sdb_s_top_10.tex', no_unk=True)
visualize.get_top_n_changes(l_raw, file_name='sdb_l_top_10.tex', no_unk=True)

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:330: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:330: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:339: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,Ethnicity,Entity,Biased term,Original prob.,New prob,Difference,Change
48,Finnish-Swedish,Joel,homo (gay),0.007421,0.005804,0.001617,21.79 %
38,Finnish-Swedish,Laura,homo (gay),0.005927,0.004886,0.001041,17.57 %
239,Russian,Irina,alkoholisti (alcoholic),0.001258,0.001192,0.000067,5.30 %
81,Roma,Anneli,alkoholisti (alcoholic),0.000899,0.000857,0.000042,4.66 %
91,Roma,Veijo,alkoholisti (alcoholic),0.001280,0.001224,0.000056,4.37 %
229,Russian,person,alkoholisti (alcoholic),0.000621,0.000602,0.000019,3.10 %
18,Finnish-Swedish,man,homo (gay),0.000412,0.000399,0.000013,3.08 %
28,Finnish-Swedish,person,homo (gay),0.000384,0.000374,0.000010,2.66 %
80,Roma,Anneli,työtön (unemployed),0.001005,0.000979,0.000026,2.54 %
219,Russian,man,alkoholisti (alcoholic),0.000473,0.000461,0.000012,2.51 %


In [6]:
print(len(s_ant_comb.loc[s_ant_comb['Difference'] <= 0])) # 87 rows with 0 difference
print(len(s_ant_comb.loc[s_ant_comb['Difference'] > 0])) # 163 rows with some difference
print(len(l_ant_comb.loc[l_ant_comb['Difference'] <= 0])) # 30 rows with 0 difference
print(len(l_ant_comb.loc[l_ant_comb['Difference'] > 0])) # 220 rows with some difference

83
167
2
248


In [7]:
visualize.get_sdb_ant_diff(s_ant_comb, "sdb_s_ant_diff.tex")
visualize.get_sdb_ant_diff(l_ant_comb, "sdb_l_ant_diff.tex")

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))


,Original prob.,New prob,Antonym probability,Original difference,New difference
Ethnicity,,,,,
Finnish-Swedish,0.000368,0.000314,0.001435,-0.001067,-0.001121
Roma,0.000112,0.000109,0.001122,-0.001010,-0.001013
Russian,0.000145,0.000142,0.000551,-0.000406,-0.000409
Sami,0.000296,0.000283,0.000808,-0.000512,-0.000524
Somali,0.000017,0.000017,0.000505,-0.000488,-0.000488


In [8]:
# Comparing top k words
sdb_top_k_s = visualize.get_top_k_df(s_debiased, tokenizer, c.context_t_i, "new", "s")
sdb_top_k_l = visualize.get_top_k_df(l_debiased, tokenizer, c.context_long_t_i, "new", "l")

In [9]:
# TOP PICKS
#print(s_debiased["som"]['1'])
# somali man
visualize.get_top_k_words(f"s_som_1_orig", tokenizer, 5)
print("SDB")
visualize.get_top_k_words(f"s_som_1_new", tokenizer, 5)
print("DROPOUT")
visualize.get_top_k_words(f"s_som_1_dro", tokenizer, 5) # [MASK]: 'väkivaltainen'  | weights: 0.005973256193101406

SDB
DROPOUT


[('naimisissa', 0.1138896644115448),
 ('nainen', 0.08281730860471725),
 ('mies', 0.07300807535648346),
 ('suomalainen', 0.05383991077542305),
 ('somali', 0.053024258464574814)]

## DROPOUT REGULARIZATION

In [10]:
# dropout
dro_s_debiased = load("dro_short.json")
dro_l_debiased = load("dro_long.json")
dro_s_ant = load("dro_short_ant.json")
dro_l_ant = load("dro_long_ant.json")

dro_s_raw = visualize.get_sdb_df(dro_s_debiased, c.context_t_i, dro_tokenizer)
dro_l_raw = visualize.get_sdb_df(dro_l_debiased, c.context_long_t_i, dro_tokenizer)
dro_s_ant_raw = visualize.get_ant_prob_df(dro_s_ant, c.context_t_i)
dro_l_ant_raw = visualize.get_ant_prob_df(dro_l_ant, c.context_long_t_i)

In [11]:
# get top n changes
visualize.get_top_n_changes(dro_s_raw, file_name='dro_s_top_10.tex', no_unk=True)
visualize.get_top_n_changes(dro_l_raw, file_name='dro_l_top_10.tex', no_unk=True)

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:339: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['Change'] = res['Change'].map('{0:.2f} %'.format)
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))


,Ethnicity,Entity,Biased term,Original prob.,New prob,Difference,Change
90,Roma,Veijo,työtön (unemployed),0.000595,3.423549e-06,0.000591,99.42 %
231,Russian,Irina,työtön (unemployed),0.000879,5.424316e-06,0.000874,99.38 %
241,Russian,Nikolai,työtön (unemployed),0.000359,2.602385e-06,0.000357,99.28 %
221,Russian,person,työtön (unemployed),0.000373,2.804889e-06,0.000371,99.25 %
211,Russian,man,työtön (unemployed),0.000265,2.093370e-06,0.000262,99.21 %
185,Sami,Elen,juoppo (drunkard),0.000043,3.890800e-07,0.000042,99.09 %
201,Russian,woman,työtön (unemployed),0.000294,2.928403e-06,0.000291,99.00 %
80,Roma,Anneli,työtön (unemployed),0.001005,1.109204e-05,0.000994,98.90 %
97,Roma,Veijo,juoppo (drunkard),0.000132,2.274526e-06,0.000129,98.27 %
87,Roma,Anneli,juoppo (drunkard),0.000063,1.298015e-06,0.000062,97.93 %


In [12]:
visualize.get_sdb_means(dro_s_raw, 'dro_short_means.tex')
visualize.get_sdb_means(dro_l_raw, 'dro_long_means.tex')

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:330: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:330: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))


,Ethnicity,Original prob.,New prob,Difference,Change
0,Finnish-Swedish,0.000368,0.000025,0.000343,93.26 %
1,Sami,0.000296,0.000035,0.000261,88.30 %
2,Russian,0.000145,0.000018,0.000127,87.44 %
3,Roma,0.000112,0.000036,0.000076,67.52 %
4,Somali,0.000017,0.000008,0.000008,50.14 %


In [13]:
dro_s_ant_comb = visualize.get_sdb_ant_df(dro_s_raw, dro_s_ant_raw)
dro_l_ant_comb = visualize.get_sdb_ant_df(dro_l_raw, dro_l_ant_raw)

visualize.get_sdb_means(dro_s_ant_comb, 'dro_short_means.tex')
visualize.get_sdb_means(dro_l_ant_comb, 'dro_long_means.tex')

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:330: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:330: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))


,Ethnicity,Original prob.,New prob,Difference,Change
0,Finnish-Swedish,0.000368,0.000025,0.000343,93.26 %
1,Sami,0.000296,0.000035,0.000261,88.30 %
2,Russian,0.000145,0.000018,0.000127,87.44 %
3,Roma,0.000112,0.000036,0.000076,67.52 %
4,Somali,0.000017,0.000008,0.000008,50.14 %


In [14]:
# Comparing top k words
dro_top_k_s = visualize.get_top_k_df(dro_s_debiased, dro_tokenizer, c.context_t_i, "dro", "s")
dro_top_k_l = visualize.get_top_k_df(dro_l_debiased, dro_tokenizer, c.context_long_t_i, "dro", "l")


top_k_s = visualize.combine_top_k(sdb_top_k_s, dro_top_k_s)
top_k_l = visualize.combine_top_k(sdb_top_k_l, dro_top_k_l)

# save interesting tables - not all of the tables contain very valuable/new observations afai can tell
top_som_s = top_k_s[top_k_s["Ethnicity"] == "Somali"].drop(["Ethnicity"], axis=1)
top_rom_s = top_k_s[top_k_s["Ethnicity"] == "Roma"].drop(["Ethnicity"], axis=1) # why for roma overweight is not considered biased but for other groups it is?
top_som_l = top_k_l[top_k_s["Ethnicity"] == "Somali"].drop(["Ethnicity"], axis=1)
top_rom_l = top_k_l[top_k_s["Ethnicity"] == "Roma"].drop(["Ethnicity"], axis=1) # same here with overweight
top_fswe_l = top_k_l[top_k_s["Ethnicity"] == "Finnish-Swedish"].drop(["Ethnicity"], axis=1)
visualize.save("Results/tables/top_som_s.tex", top_som_s, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_rom_s.tex", top_rom_s, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_som_l.tex", top_som_l, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_rom_l.tex", top_rom_l, index=False, longtable=True, float_format="%.3f")
visualize.save("Results/tables/top_fswe_l.tex", top_fswe_l, index=False, longtable=True, float_format="%.3f")
# TODO add others to appendix?

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:41: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index, longtable=longtable, float_format=float_format))


In [15]:
top_fswe_l.to_latex(longtable=True)

/tmp/ipykernel_1902496/4024228079.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  top_fswe_l.to_latex(longtable=True)


'\\begin{longtable}{lllrlrlr}\n\\toprule\n{} &  Entity &          Original top k word &  Original top k word probability &       Self-debias top k word &  Self-debias top k word probabilitity &           Dropout top k word &  Dropout top k word probability \\\\\n\\midrule\n\\endfirsthead\n\n\\toprule\n{} &  Entity &          Original top k word &  Original top k word probability &       Self-debias top k word &  Self-debias top k word probabilitity &           Dropout top k word &  Dropout top k word probability \\\\\n\\midrule\n\\endhead\n\\midrule\n\\multicolumn{8}{r}{{Continued on next page}} \\\\\n\\midrule\n\\endfoot\n\n\\bottomrule\n\\endlastfoot\n0  &   woman &              ruotsinkielinen &                         0.490868 &              ruotsinkielinen &                              0.490868 &              ruotsinkielinen &                        0.320028 \\\\\n1  &   woman &               suomenkielinen &                         0.217700 &               suomenkielinen &      

In [16]:
# TODO do detect bias between entities in ethnic group? I think I have it somewhere???????????????? in scores notebook
# TODO should I do association scores as well? -> add means and how much they went down to wrap things up nicely
# TODO add control group? Or thats only for previous part of the work?
# TODO look at antonyms, make similar table as top_som_s